## Sakila DB

https://www.jooq.org/sakila

In [ ]:
SELECT * FROM staff;

In [ ]:
SELECT * FROM inventory;

In [ ]:
SELECT f.title, count(*) as num_rentals
FROM inventory i 
	LEFT JOIN film f ON i.film_id = f.film_id
	LEFT JOIN rental r ON r.inventory_id = i.inventory_id
GROUP BY f.title
ORDER BY num_rentals desc
LIMIT 5; 

## SQL Re-Intro

As data analysts, we will often rely on a team of technologists to maintain and setup a database. 
Since data analysts are mostly interested in “interrogating” a dataset, our focus often lies only on using query language statements to generate reports & metrics.
Ex:
* Write a script that calculates sales for the month 💰
* Write a script that gives me the last 5 employees that clocked out for the day  😨

But just in case, we will also go over the theory & practice of designing a database, as well as a couple of advanced features that will make you a more powerful developer.  
First, let’s review the basics.

## Definitions

Database Management System: Software tool for storing & managing large amounts of data (Postgres, MySQL, Oracle). Allows access via query language. Allows for durable (failure-proof) and atomic operations.  

Database Server: An installation of a DBMS. We have a database server on Amazon, and our dbms is Postgres  

Database: Collection of data organized for a specific application  

Database Application: Software product that uses a DBMS (pgAdmin)

Relational Model: Approach to managing data using a structure where all data is represented as tuples (rows), grouped into relations (tables)  

Database Schema: A logical plan for a database, also a structure that represents a subset of functions & tables w/in a database  

Primary Key: A set of attributes in a relation that identifies a specific tuple   

Foreign Key: A referential integrity constraint, a link between two tables  

## Transactions

**ACID**

* Atomicity: All changes are performed or NONE.
* Consistency: Data is not lost or unpredictably changed
* Isolation: Intermediate state of transaction is not visible to other transactions
* Durability: After a transaction completes, changes to data are persistent 

**Syntax** 
COMMIT save changes to database  
ROLLBACK undo changes to database  
SAVEPOINT create a point to rollback to in database  
SET TRANSACTION name the transaction  

Creating transactions allow us to undo unfortunate & unintended queries. This will become relevant again next week in sqlalchemy. 

**PostgreSQL**
Postgres automatically treats each DDL & DML query as a transaction. However, we could implement further functionality to our queries by utilizing:

* BEGIN; … COMMIT;  (Begin transaction, and commit if successful)
* SAVEPOINT name;   (Set a save-point to rollback to)
* ROLLBACK; 	  (Rollback to that save-point)


In [ ]:
SELECT * FROM inventory WHERE inventory_id = 4581;

In [ ]:
BEGIN;
	SAVEPOINT undo_change;
	UPDATE inventory SET store_id = 2 WHERE inventory_id = 4581;
	-- woops, undo
	ROLLBACK TO undo_change;
COMMIT;

As you can imagine, this becomes super-useful if you are working on a database that can have multiple changes occuring to it from multiple sources. By ensuring ACID properties, we will not accidentally make an unintended (and untraceable) change to our database which will compromise the entire operation!

Notice that a bad transaction will result in the error message:

`ERROR: current transaction is aborted, commands ignored until end of transaction block`

In [ ]:
BEGIN;
	SAVEPOINT undo_change;
	UPDATE inventory SET store_id = 5 WHERE inventory_id = 4581;
	-- woops, undo
	ROLLBACK;
COMMIT;

Forcing an error causes us to stay in the same "broken" transaction. We must rollback to begin a new transaction. This highlights the importance of exception handling, which will also come up during sqlalchemy. 

Unfortunately, there is no trivial way to catch these exceptions in a SQL script. An additional wrapper of Python therefore would give us more functionality.

In [ ]:
ROLLBACK;

BEGIN;
	SAVEPOINT undo_change;
	UPDATE inventory SET store_id = 2 WHERE inventory_id = 4581;
	-- woops, undo
	ROLLBACK;
COMMIT;

## PL/pgSQL

Outside of regular queries, we also have the ability to generate procedural (programming-like) queries that utilize control flow structures such as conditionals, loops, and functions.
This is a block structured language that must abide by the following format:

```
DECLARE
...
BEGIN
...
END
```

This must be wrapped in anonymous function, or a named function.

Note: you cannot put transactions inside of procedural SQL! 

**Anonymous Function**

In [ ]:
DO $$
DECLARE
    word VARCHAR := 'Hello World';
    today TIMESTAMP;
BEGIN
    SELECT CURRENT_DATE INTO today;
    -- PL/pgSQL conditional
    IF today= DATE '2023-04-18' THEN
        RAISE NOTICE '%', word;
    END IF;
    --
END $$;

In [ ]:
DO $$
DECLARE
	cnt INT;
    word VARCHAR := 'Hello World';
    today TIMESTAMP;
BEGIN
    SELECT CURRENT_DATE INTO today;
    IF today= DATE '2023-04-18' THEN
        -- PL/pgSQL loop
        FOR cnt IN 1..5 LOOP
            RAISE NOTICE '%', word;
        END LOOP;
        --
    END IF;
END $$;

**Named Function**

In [ ]:
-- create a function named 'func1'. Replace it if it exists already. It returns a varchar
CREATE OR REPLACE FUNCTION func1 (param1 VARCHAR) RETURNS VARCHAR AS $$ 
    -- declare variables that you will use in your function
    DECLARE
    	today TIMESTAMP;
    -- begin the procedure
    BEGIN
        -- basic sql query
        SELECT CURRENT_DATE INTO today;
        -- control flow
		IF today= DATE '2023-04-18' THEN
			RAISE NOTICE '%', param1;
		END IF;
        -- return value
        RETURN 'hello world!';
    -- end procedure
    END; $$ 
-- specify language 
LANGUAGE plpgsql;


SELECT func1('me')

Notice that the `$$` is just syntactic sugar that allows us to create procedures. We can replace this with a `'`, but then we would have to apply additional formatting to our function.

In [ ]:
CREATE OR REPLACE FUNCTION func1 (param1 VARCHAR) RETURNS VARCHAR AS '
    DECLARE
    	today TIMESTAMP;
    BEGIN
        SELECT CURRENT_DATE INTO today;
		IF today= DATE ''2023-04-18'' THEN
			RAISE NOTICE ''%'', param1;
		END IF;
        RETURN ''hello world!'';
    END ' 
LANGUAGE plpgsql;

SELECT func1('me')

## Cursors

An encapsulation of a query. Reads a few rows at a time instead of the entire result. Great for queries that result in a large amount of data.  
Comes up in sqlalchemy again! 

"Read-only pointer that allows a program, regardless of the language used, to access the result set of a query."

Can only exist within a procedure.


https://kb.objectrocket.com/postgresql/what-is-a-postgresql-cursor-749 

In [ ]:
CREATE OR REPLACE FUNCTION cursor_func () RETURNS VARCHAR AS $$ 
    DECLARE 
        -- create cursor
        curs1 CURSOR FOR SELECT * FROM film;
        -- create row var
        film_row film%ROWTYPE;
    BEGIN
        -- open cursor for manipulation
        OPEN curs1;
        /* get next result row */
        FETCH curs1 INTO film_row;
        RETURN film_row.title;
		CLOSE curs1;
	END; $$ 
LANGUAGE plpgsql;

SELECT cursor_func()

## Basic Queries

Since SQL is a declarative language, we do not get to specify how a piece of code is executed.

However, we should still be aware as to what is going on underneath the hood of SQL.

Data in SQL is stored in fundamental data units called 8 KB “pages”

1. Queries are passed to a Query Evaluation Engine
* Parses query for syntax error
* Optimizes query
* Executes query
2. File Manager: checks if user can access data, asks buffer to find page
3. Buffer Manager: checks to see if data is in buffer, if not asks disk manager
4. Disk Manager: accesses page data 

**Execution Order**

1. Row Filtering - Phase 1: 
* FROM, 
* WHERE,
* GROUP BY
2. Column Filtering: 
* SELECT
3. Row Filtering - Phase 2:
* DISTINCT
* ORDER BY
* LIMIT

In [ ]:
-- basic
SELECT *
FROM film;

-- limit
SELECT *
FROM film
LIMIT 5;

-- aggregate
SELECT min(release_year)
FROM film;

-- existence
SELECT 1
FROM film
WHERE release_year=2005;

-- null
SELECT first_name
FROM staff
WHERE picture is NULL;

-- pattern
SELECT first_name
FROM staff
WHERE first_name LIKE 'M%';

-- group by's
SELECT rating, count(*)
FROM film
GROUP BY rating;

-- order by
SELECT rating, count(*) as a
FROM film
GROUP BY rating
ORDER BY a desc;

-- quantifier/correlated subquery
SELECT f.title, f.language_id
FROM film f WHERE EXISTS  (
SELECT 1 FROM language l WHERE l.language_id = 3

-- cartesian product
SELECT f.title, l.name
FROM film f, language l
WHERE l.language_id = f.language_id;

-- joins
SELECT f.title, l.name
FROM film f INNER JOIN language l
ON l.language_id = f.language_id;


## Views 

Virtual tables that are created from query language commands.

Creating views allows us to:
* Hide complexity
* Show different tables for different users
* Allow updates to original table from view  (based on assumptions!)

Anonymous views

In [ ]:
SELECT * 
FROM  ( SELECT title, length FROM film WHERE rating='PG') as anon_view
WHERE anon_view.length > 100; 

Regular views

In [ ]:
CREATE VIEW pg_film(title, film)
AS 
SELECT title, length FROM film WHERE rating='PG';

## Window Functions

Allow us to compute aggregates without a GROUP BY! 

https://www.postgresqltutorial.com/postgresql-window-function/ 

https://www.postgresql.org/docs/current/tutorial-window.html 

Super useful for SQL interview questions. E.g. What is(are) the title of second longest film(s)?

```sql
SELECT iq.title FROM
	( SELECT title, dense_rank() OVER (ORDER BY length desc) as rnk
		FROM film 
	 ) as iq
WHERE rnk = 2
```

In [ ]:
SELECT title, rating, avg(length) OVER (PARTITION BY rating) as avg_length 
FROM film;

## Database Design

Data Modeling: The process of creating a data model in relational algebra to store data in a database.   

Conceptual Data Model: WHAT does the system contain. Typically assisted by business stakeholders (what data do we want to track?)  

Logical Data Model: HOW should the system store the data. Expressed as an ERD. Typically assisted by data architects.  

Physical Data Model: your schema    

## Conceptual Database

1. Purpose: what would be the purpose of my database?

2. Information: what information do I have already?

* Identify Objects (aka entities): 
* Out of the information that I already have, what columns do I want to keep? 
* Do subsets of these columns represent different objects? 
    * Product, Customer, Store
    * Company, Job

* Do these objects relate to one another at all?


Keep in mind, these are concepts & ideation. We will formalize these in the next step.

## Logical Database

Formalize these observations via an entity relationship diagram. We will be using draw.io 

Using your concepts, create a graph that describes how your database will look.

## Keys

Choosing a Primary Key
* Key: A unique identifier of that row. Cannot be reused by any other row. Keep in mind that keys can point to rows with the same data, but keys cannot be reused! 
* Cannot be null! Can be composed of multiple columns.

* Aim to use existing attributes. However, if no unique identifier exists, make up an id col.
* Underline your primary key attribute in your ERD.

Including Foreign Keys
* If your table refers to information within another table, you should also create a foreign key.

## Functional Dependency

One (or more) attribute(s) uniquely determines another (group of ) attribute(s).
R is a relation with attributes X & Y. We state there is a functional dependency b/w X & Y (X -> Y) if each value of  X is associated with precisely one Y value. In this case, X is the determinant  and Y is the dependent attribute.

Only applies to tuples. 

Given the value of X we can determine information in the rest of the row, but not vice-versa.

Ex:
```
Order_id -> (port_id, type_id)
Employee_id -> (name, address, salary)
```

https://www.techopedia.com/definition/19504/functional-dependency 

**Normalization**

**1NF**  
Attributes do not contain relations (data structures) as elements  
**2NF**  
All non-key attributes are fully functional dependent on the primary key. Create separate tables for sets of values that apply to multiple tables.  
**3NF**   
No transitive dependency. Eliminate all fields that do not depend on the primary key.  
**BCNF**   
Often not practical to apply. Determinant is a super-key.  

https://arctype.com/blog/2nf-3nf-normalization-example/ 
https://learn.microsoft.com/en-us/office/troubleshoot/access/database-normalization-description 